## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("C:/Analysis Projects/World_Weather_Analysis/Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Cabedelo,-6.9811,-34.8339,84.87,71,56,13.20,BR,2022-04-02 19:14:33
1,1,Bambous Virieux,-20.3428,57.7575,75.22,84,100,18.37,MU,2022-04-02 19:14:33
2,2,Ushuaia,-54.8000,-68.3000,40.66,93,100,4.61,AR,2022-04-02 19:14:34
3,3,Mayor Pablo Lagerenza,-19.9309,-60.7718,85.87,35,17,6.55,PY,2022-04-02 19:14:34
4,4,Nishihara,26.1842,127.7558,63.68,78,100,8.05,JP,2022-04-02 19:14:34


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Cabedelo,-6.9811,-34.8339,84.87,71,56,13.20,BR,2022-04-02 19:14:33
1,1,Bambous Virieux,-20.3428,57.7575,75.22,84,100,18.37,MU,2022-04-02 19:14:33
6,6,Road Town,18.4167,-64.6167,80.64,65,75,10.36,VG,2022-04-02 19:14:35
13,13,Rikitea,-23.1203,-134.9692,74.14,56,28,6.67,PF,2022-04-02 19:14:38
15,15,Bonthe,7.5264,-12.5050,76.46,89,100,13.22,SL,2022-04-02 19:14:38
19,19,Saint-Philippe,-21.3585,55.7679,74.79,97,100,21.50,RE,2022-04-02 19:14:40
29,29,Avarua,-21.2078,-159.7750,80.65,83,96,3.44,CK,2022-04-02 19:14:43
30,30,Dusi,12.7667,79.6833,78.12,87,6,4.76,IN,2022-04-02 19:14:43
32,32,Hilo,19.7297,-155.0900,75.25,89,100,5.75,US,2022-04-02 19:14:44
33,33,Pitimbu,-7.4706,-34.8086,84.79,69,30,12.01,BR,2022-04-02 19:14:44


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID       220
City          220
Lat           220
Lng           220
Max Temp      220
Humidity      220
Cloudiness    220
Wind Speed    220
Country       220
Date          220
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna

<bound method DataFrame.dropna of      City_ID             City      Lat       Lng  Max Temp  Humidity  \
0          0         Cabedelo  -6.9811  -34.8339     84.87        71   
1          1  Bambous Virieux -20.3428   57.7575     75.22        84   
6          6        Road Town  18.4167  -64.6167     80.64        65   
13        13          Rikitea -23.1203 -134.9692     74.14        56   
15        15           Bonthe   7.5264  -12.5050     76.46        89   
..       ...              ...      ...       ...       ...       ...   
635      635             Daru  -9.0763  143.2092     77.14        83   
637      637           Mapiri -15.2500  -68.1667     77.94        65   
638      638           Maceio  -9.6658  -35.7353     83.64        65   
646      646       Brokopondo   4.7500  -55.1667     78.01        89   
649      649             Boda   4.3189   17.4695     79.38        60   

     Cloudiness  Wind Speed Country                 Date  
0            56       13.20      BR  2022-

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Cabedelo,BR,84.87,-6.9811,-34.8339,
1,Bambous Virieux,MU,75.22,-20.3428,57.7575,
6,Road Town,VG,80.64,18.4167,-64.6167,
13,Rikitea,PF,74.14,-23.1203,-134.9692,
15,Bonthe,SL,76.46,7.5264,-12.5050,
19,Saint-Philippe,RE,74.79,-21.3585,55.7679,
29,Avarua,CK,80.65,-21.2078,-159.7750,
30,Dusi,IN,78.12,12.7667,79.6833,
32,Hilo,US,75.25,19.7297,-155.0900,
33,Pitimbu,BR,84.79,-7.4706,-34.8086,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
# 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [14]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Cabedelo,BR,84.87,-6.9811,-34.8339,Hotel Pigalle
1,Bambous Virieux,MU,75.22,-20.3428,57.7575,Casa Tia Villa
6,Road Town,VG,80.64,18.4167,-64.6167,"ZINGARA 76ft Catamaran, full crew-all included"
13,Rikitea,PF,74.14,-23.1203,-134.9692,People ThankYou
15,Bonthe,SL,76.46,7.5264,-12.5050,Bonthe Holiday Village
19,Saint-Philippe,RE,74.79,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
29,Avarua,CK,80.65,-21.2078,-159.7750,Paradise Inn
30,Dusi,IN,78.12,12.7667,79.6833,HOTEL RAHMANIA
32,Hilo,US,75.25,19.7297,-155.0900,Hilo Hawaiian Hotel
33,Pitimbu,BR,84.79,-7.4706,-34.8086,Reserva do Abiaí


In [15]:
# 8a. Create the output File (CSV)
output_data_file = "C:/Analysis Projects/World_Weather_Analysis/Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))